In [1]:
import numpy as np
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# generate some example data
X = np.random.rand(100, 10)
y = np.random.rand(100, 2)

# define the range of alpha values to test for each subset
alpha_range1 = np.logspace(-4, 4, 9)
alpha_range2 = np.logspace(-2, 2, 5)

# define the cross-validation scheme
kf = KFold(n_splits=5)

# define the MultiTaskElasticNet model
enet = MultiTaskElasticNet()

# define the grid search parameters
param_grid = {'alpha': [alpha_range1, alpha_range2], 'l1_ratio': [0.1, 0.5, 0.9]}

# perform the grid search using cross-validation
best_score = -np.inf
for alpha1 in alpha_range1:
    for alpha2 in alpha_range2:
        for l1_ratio in param_grid['l1_ratio']:
            enet.set_params(alpha=[alpha1, alpha2], l1_ratio=l1_ratio)
            scores = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                X_val, y_val = X[val_idx], y[val_idx]
                enet.fit(X_train, y_train)
                score = enet.score(X_val, y_val)
                scores.append(score)
            mean_score = np.mean(scores)
            if mean_score > best_score:
                best_score = mean_score
                best_params = {'alpha1': alpha1, 'alpha2': alpha2, 'l1_ratio': l1_ratio}

# print the best hyperparameters and the corresponding mean score
print("Best alpha1 value:", best_params['alpha1'])
print("Best alpha2 value:", best_params['alpha2'])
print("Best l1_ratio value:", best_params['l1_ratio'])
print("Best mean score:", best_score)

InvalidParameterError: The 'alpha' parameter of MultiTaskElasticNet must be a float in the range [0, inf). Got [0.0001, 0.01] instead.

In this updated example, we define a range of gamma values to test (i.e., the hyperparameter for the second L2 penalty), and we add 'gamma' to the param_grid dictionary. We then use a nested for loop to iterate through all combinations of alpha and gamma.

Inside the nested loop, we use Ridge.set_params() to set the hyperparameters of the Ridge Regression model, and we use KFold.split() to split the data into training and validation sets for cross-validation. We then fit the Ridge Regression model on the training set and calculate the validation score. We repeat this process for each fold of the cross-validation and take the mean of the validation scores as the final score for this combination of hyperparameters.

Finally, we check if the current score is better than the previous best score, and if so, we update the best_score and best_params variables. At the end of the loop, we print the best hyperparameters and corresponding mean score.

In [7]:
import numpy as np
# import glum
from glum import GeneralizedLinearRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold

# generate some example data
X = np.random.rand(100, 10)
y = np.random.rand(100, 2)

# define the range of gamma values to test for each subset
gamma_range1 = np.logspace(-4, 4, 9)
gamma_range2 = np.logspace(-2, 2, 5)

# define the cross-validation scheme
# cv = glum.CrossValidation(n_splits=5)
cv = KFold(n_splits=5)

# define the GeneralizedLinearRegressor model with ElasticNet penalty
reg = GeneralizedLinearRegressor(P2=ElasticNet())

# define the grid search parameters
param_grid = {'gamma': [gamma_range1, gamma_range2], 'l1_ratio': [0.1, 0.5, 0.9]}

# perform the grid search using cross-validation
best_score = -np.inf
for gamma1 in gamma_range1:
    for gamma2 in gamma_range2:
        for l1_ratio in param_grid['l1_ratio']:
            reg.penalty = ElasticNet(alpha=[gamma1, gamma2], l1_ratio=l1_ratio)
            scores = []
            for train_idx, val_idx in cv.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                X_val, y_val = X[val_idx], y[val_idx]
                reg.fit(X_train, y_train)
                score = reg.score(X_val, y_val)
                scores.append(score)
            mean_score = np.mean(scores)
            if mean_score > best_score:
                best_score = mean_score
                best_params = {'gamma1': gamma1, 'gamma2': gamma2, 'l1_ratio': l1_ratio}

# print the best hyperparameters and the corresponding mean score
print("Best gamma1 value:", best_params['gamma1'])
print("Best gamma2 value:", best_params['gamma2'])
print("Best l1_ratio value:", best_params['l1_ratio'])
print("Best mean score:", best_score)

ValueError: y should be a 1d array, got an array of shape (80, 2) instead.

In [ ]:
import numpy as np

def cross_validated_ridge_regression(X, y, n_folds, penalty_params_list):
    """
    Performs cross-validated ridge regression with multiple L2 penalization parameters.
    
    Args:
    - X (ndarray): A numpy array of shape (n_samples, n_features) containing the input data.
    - y (ndarray): A numpy array of shape (n_samples, n_targets) containing the target data.
    - n_folds (int): The number of folds to use in cross-validation.
    - penalty_params_list (list of tuples): A list of tuples specifying the penalty parameters to use for each subset
        of features. Each tuple should have two elements: a numpy array of boolean values indicating which features to
        include, and a float specifying the corresponding penalty parameter for those features.
    
    Returns:
    - best_params (dict): A dictionary containing the best penalty parameters found by cross-validation for each subset
        of features. The keys are the indices of the subsets (starting from 0), and the values are the corresponding
        penalty parameters.
    - best_score (float): The mean cross-validation score achieved by the best set of penalty parameters.
    """
    
    # initialize variables
    n_samples, n_features = X.shape
    n_targets = y.shape[1]
    scores = np.zeros((len(penalty_params_list), n_folds))
    best_params = {}
    best_score = -np.inf
    
    # create indices for cross-validation
    fold_indices = np.repeat(np.arange(n_folds), n_samples // n_folds + 1)[:n_samples]
    
    # loop over the subsets of features and their corresponding penalty parameters
    for i, (feature_indices, penalty_param) in enumerate(penalty_params_list):
        
        # create the design matrix for the current subset of features
        X_subset = X[:, feature_indices]
        
        # compute the Gram matrix for ridge regression
        G = np.dot(X_subset.T, X_subset) + penalty_param * np.eye(X_subset.shape[1])
        
        # loop over the folds
        for fold in range(n_folds):
            
            # get the indices for the current fold
            train_indices = (fold_indices != fold)
            test_indices = (fold_indices == fold)
            
            # split the data into training and test sets
            X_train = X_subset[train_indices, :]
            y_train = y[train_indices, :]
            X_test = X_subset[test_indices, :]
            y_test = y[test_indices, :]
            
            # compute the ridge regression coefficients using the training set
            w = np.linalg.solve(G, np.dot(X_train.T, y_train))
            
            # compute the mean squared error on the test set
            y_pred = np.dot(X_test, w)
            mse = np.mean((y_pred - y_test)**2)
            
            # store the score for this fold
            scores[i, fold] = -mse
        
        # compute the mean score over all folds for this set of penalty parameters
        mean_score = np.mean(scores[i, :])
        
        # update the best score and best parameters if necessary
        if mean_score > best_score:
            best_score = mean_score
            best_params[i] = penalty_param
    
    return best_params, best_score
